In [1]:
import numpy as np
import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt 
import matplotlib.style as stl 
from tensorflow import keras 
from keras.preprocessing.text import Tokenizer
from nltk.corpus import stopwords
import string
from tensorflow.keras.models import Sequential,Model
from tensorflow.keras.layers import Dense, LSTM, Embedding,Dropout,SpatialDropout1D,Conv1D,MaxPooling1D,GRU,BatchNormalization
import warnings
warnings.filterwarnings('ignore')
stl.use('ggplot')
from tqdm import tqdm
import string
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.stem.snowball import SnowballStemmer
import regex as re

In [2]:
data=pd.read_csv("../input/samplcsv/sample-2.csv")

In [3]:
newdata=pd.read_csv("../input/oathss/oaths.csv")

In [4]:
data["transcription_text"][10]

'[[image - eagle]]\r\nAMNESTY OATH.\r\n\r\nMississippi City Miss Sept 29 1865.\r\n\r\nI, J.W. Baxter, do solemnly swear, in presence of Almighty God, that I will henceforth faithfully support, protect and defend the Constitution of the United States and the union of the States thereunder; and that I will, in like manner, abide by and faithfully support all laws and proclamations which have been made during the existing rebellion with reference to the emancipation of slaves. SO HELP ME GOD.\r\nJ.W. Baxter\r\n\r\nSworn to and subscribed at Mississippi City\r\nthis 29th day of September 1865, before me\r\nJ. H. Mathews Capt 66th\r\nU.S Col Infty & Provost Marshal.  Sub port Miss\r\n\r\nDESCRIPTION:\r\nAge 24 years\r\nHeight 5ft 8 inches\r\nEyes Blue\r\nColor of Hair light\r\nCharacteristics [[blank]]\r\n\r\nHEADQUARTERS, Mississippi City Miss\r\nSept 29th 1865.\r\n\r\nI hereby certify, that the Amnesty Oath prescribed by the Proclamation of the President of the United States, was duly adm

In [5]:
txte= re.sub("([\(\[]).*?([\)\]])","",data["transcription_text"][10])
txte

']\r\nAMNESTY OATH.\r\n\r\nMississippi City Miss Sept 29 1865.\r\n\r\nI, J.W. Baxter, do solemnly swear, in presence of Almighty God, that I will henceforth faithfully support, protect and defend the Constitution of the United States and the union of the States thereunder; and that I will, in like manner, abide by and faithfully support all laws and proclamations which have been made during the existing rebellion with reference to the emancipation of slaves. SO HELP ME GOD.\r\nJ.W. Baxter\r\n\r\nSworn to and subscribed at Mississippi City\r\nthis 29th day of September 1865, before me\r\nJ. H. Mathews Capt 66th\r\nU.S Col Infty & Provost Marshal.  Sub port Miss\r\n\r\nDESCRIPTION:\r\nAge 24 years\r\nHeight 5ft 8 inches\r\nEyes Blue\r\nColor of Hair light\r\nCharacteristics ]\r\n\r\nHEADQUARTERS, Mississippi City Miss\r\nSept 29th 1865.\r\n\r\nI hereby certify, that the Amnesty Oath prescribed by the Proclamation of the President of the United States, was duly administered to J.W. Baxter

# Preprocessing

In [6]:
text=data["transcription_text"]

In [7]:
all_text=newdata["transcription_text"]

In [8]:
stop_words = set(stopwords.words('english'))
punctuation = string.punctuation
stemmer = SnowballStemmer(language="english")
lemmatizer = WordNetLemmatizer()

In [9]:
stop_words.remove("of")
punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [10]:
def process(s):
    s= re.sub("([\(\[]).*?([\)\]])"," ",s)
    for p in punctuation:
        s = s.replace(p, '')
    s = word_tokenize(s)
    s = [w for w in s if not w in stop_words] #optional
    s = [lemmatizer.lemmatize(word) for word in s]
    return s

In [11]:
for i in tqdm(range(data.shape[0])):
    text[i] = process(text[i])

100%|██████████| 20/20 [00:01<00:00, 10.88it/s]


In [12]:
for i in tqdm(range(newdata.shape[0])):
    all_text[i] = process(all_text[i])

100%|██████████| 2136/2136 [00:04<00:00, 436.63it/s]


In [13]:
for i in tqdm(range(data.shape[0])):
    text[i] = " ".join(text[i])

100%|██████████| 20/20 [00:00<00:00, 3515.03it/s]


In [14]:
for i in tqdm(range(newdata.shape[0])):
    all_text[i] = " ".join(all_text[i])

100%|██████████| 2136/2136 [00:00<00:00, 2554.71it/s]


In [15]:
all_text

0       I I C Meekins of Currituck County State of Nor...
1       Department of Virginia North Carolina 1864 tak...
2       I W H Pearce of Craven County State of NorthCa...
3       State of North Carolina County I solemnly swea...
4       State of North Carolina Wake County I SE Allen...
                              ...                        
2131    Office Provost Marshal Raleigh NC May 26 65 Th...
2132    July 30th 1867 Birnie Wm Agent Bu R F A L Oath...
2133    State of North Carolina Robeson County I Willi...
2134    NARA 449 102 We whose name written well person...
2135    NARA 450 404 Office Provost Marshall City of W...
Name: transcription_text, Length: 2136, dtype: object

In [16]:
text

0     8 E encl I Anne H Elliott widow solemnly swear...
1     I E C Whaley solemnly swear I carefully read A...
2     43M I Mrs E Montgomery solemnly Swear I carefu...
3     No 2324 I solemnly swear affirm presence of Al...
4     August 24 1865 APPLICATION AND CERTIFICATE OF ...
5     I Wm Whaley solemnly swear I carefully read Am...
6     I J F Steinmeyer solemnly swear I carefully re...
7     I Joseph Edings solemnly swear I carefully rea...
8     Vicksburg Miss July 22 1867 Thomas Andrew Oath...
9     455 I Jacob Schroder solemnly swear I carefull...
10    AMNESTY OATH Mississippi City Miss Sept 29 186...
11    OATH I P P Bergevin solemnly swear I never vol...
12                                         Frank Conner
13    I Caroline Belitzer solemnly Swear I carefully...
14    Triplicate AMNESTY OATH Pass Christian Miss Au...
15    HEADQUARTERS UNITED STATES FORCES No 1282 PROV...
16                                      Robert A Wilson
17    APPLICATION AND CERTIFICATE OF James D Ken

# Flair

In [17]:
!pip3 install Cython
!pip install transformers
!pip install flair

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.9/401.9 kB 1.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 6.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 21.2 MB/s eta 0:00:00
  Installing build dependencies ... - \ | / - \ done
  Getting requirements to build wheel ... - \ | / - done
  Preparing metadata (pyproject.toml) ... - \ | / - done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.3/46.3 kB 4.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - done
  Preparing metadata (setup.py) ... - done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 788.5/788.5 kB 31.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - done
  Preparing metadata (setup.py) ... - done
  Created wheel for gdown: filename=gdown-4.4.0-p

# Hugging face  : Flair

**ner-english** 

In [18]:
!git lfs install
!git clone https://huggingface.co/flair/ner-english-large

git: 'lfs' is not a git command. See 'git --help'.

The most similar command is
	log
Cloning into 'ner-english-large'...
remote: Enumerating objects: 25, done.
remote: Counting objects: 100% (25/25), done.
remote: Compressing objects: 100% (24/24), done.
remote: Total 25 (delta 10), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (25/25), 11.03 KiB | 1.38 MiB/s, done.


In [19]:
from flair.models import SequenceTagger

tagger = SequenceTagger.load("flair/ner-english-ontonotes-large")

Downloading:   0%|          | 0.00/2.24G [00:00<?, ?B/s]

2022-06-08 10:48:45,812 loading file /root/.flair/models/ner-english-ontonotes-large/2da6c2cdd76e59113033adf670340bfd820f0301ae2e39204d67ba2dc276cc28.ec1bdb304b6c66111532c3b1fc6e522460ae73f1901848a4d0362cdf9760edb1


Downloading:   0%|          | 0.00/616 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/4.83M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/8.68M [00:00<?, ?B/s]

2022-06-08 10:49:13,297 SequenceTagger predicts: Dictionary with 76 tags: <unk>, O, B-CARDINAL, E-CARDINAL, S-PERSON, S-CARDINAL, S-PRODUCT, B-PRODUCT, I-PRODUCT, E-PRODUCT, B-WORK_OF_ART, I-WORK_OF_ART, E-WORK_OF_ART, B-PERSON, E-PERSON, S-GPE, B-DATE, I-DATE, E-DATE, S-ORDINAL, S-LANGUAGE, I-PERSON, S-EVENT, S-DATE, B-QUANTITY, E-QUANTITY, S-TIME, B-TIME, I-TIME, E-TIME, B-GPE, E-GPE, S-ORG, I-GPE, S-NORP, B-FAC, I-FAC, E-FAC, B-NORP, E-NORP, S-PERCENT, B-ORG, E-ORG, B-LANGUAGE, E-LANGUAGE, I-CARDINAL, I-ORG, S-WORK_OF_ART, I-QUANTITY, B-MONEY


In [20]:
from flair.models import SequenceTagger

tagger1 = SequenceTagger.load("flair/ner-english")

Downloading:   0%|          | 0.00/432M [00:00<?, ?B/s]

2022-06-08 10:49:23,652 loading file /root/.flair/models/ner-english/4f4cdab26f24cb98b732b389e6cebc646c36f54cfd6e0b7d3b90b25656e4262f.8baa8ae8795f4df80b28e7f7b61d788ecbb057d1dc85aacb316f1bd02837a4a4
2022-06-08 10:49:25,564 SequenceTagger predicts: Dictionary with 20 tags: <unk>, O, S-ORG, S-MISC, B-PER, E-PER, S-LOC, B-ORG, E-ORG, I-PER, S-PER, B-MISC, I-MISC, E-MISC, I-ORG, B-LOC, E-LOC, I-LOC, <START>, <STOP>


In [21]:
from flair.models import SequenceTagger

tagger2 = SequenceTagger.load("flair/ner-english-large")

Downloading:   0%|          | 0.00/2.24G [00:00<?, ?B/s]

2022-06-08 10:50:16,559 loading file /root/.flair/models/ner-english-large/07301f59bb8cb113803be316267f06ddf9243cdbba92a4c8067ef92442d2c574.554244d3476d97501a766a98078421817b14654496b86f2f7bd139dc502a4f29
2022-06-08 10:50:34,883 SequenceTagger predicts: Dictionary with 20 tags: <unk>, O, S-ORG, S-MISC, B-PER, E-PER, S-LOC, B-ORG, E-ORG, I-PER, S-PER, B-MISC, I-MISC, E-MISC, I-ORG, B-LOC, E-LOC, I-LOC, <START>, <STOP>


tagger3 nerenglishlarge

In [22]:
lst=[]
label=[]
Per=[]
ORG=[]
LOC=[]
from flair.data import Sentence

dflarge=pd.DataFrame({"text":"", "PERSON-nerenglarge":[], "LOC-nerenglarge":"","ORG-nerenglarge":""})
for i in range(20):
        sentencess =Sentence(text[i])
        dflarge.loc[i,"text"]=text[i]
        tagger2.predict(sentencess)
        dc={}
        P=[]
        L=[]
        G=[]
        O=[]
        for entity in sentencess.get_spans('ner'):
                     
                     if (entity.get_label("ner").score > 0.90) & (entity.get_label("ner").value != "CARDINAL") & (entity.get_label("ner").value != "LAW")& (entity.get_label("ner").value != "QUANTITY") & (entity.get_label("ner").value != "WORK_OF_ART") & (entity.get_label("ner").value != "LAW")& (entity.get_label("ner").value != "QUANTITY") & (entity.get_label("ner").value != "EVENT")  :
                        dc[entity.text]= entity.get_label("ner").value 
                        lst.append(entity)
        
        
        for j in dc :
                if dc[j]=="PER":
                      P.append(j)
                elif (dc[j]=="ORG") | (dc[j]=="MISC") :
                      O.append(j)
                elif dc[j]=="LOC":
                      L.append(j)
               
        Per.append(P)
        ORG.append(O)
        LOC.append(L)
            

dflarge["PERSON-nerenglarge"]=Per
dflarge["LOC-nerenglarge"]=LOC
dflarge["ORG-nerenglarge"]=ORG                        

In [23]:
dflarge

,text,PERSON-nerenglarge,LOC-nerenglarge,ORG-nerenglarge
0,8 E encl I Anne H Elliott widow solemnly swear...,"[Anne H Elliott, Andrew Johnson, S Wilde Durso...","[United States of America, District of Beaufor...","[Amnesty, Constitution, Emancipation of Slaves]"
1,I E C Whaley solemnly swear I carefully read A...,"[I E C Whaley, Andrew Johnson, E C Whaley, Lev...","[United State of America, United States of Ame...","[Amnesty Proclamation, Constitution, Emancipat..."
2,43M I Mrs E Montgomery solemnly Swear I carefu...,"[Mrs E Montgomery, ANDREW JOHNSON, Sgd Montgom...","[United States of America, Charleston, SC, Cou...","[Amnesty Proclamation, 47th Pa, Vols, Emancipa..."
3,No 2324 I solemnly swear affirm presence of Al...,"[God, GOD, EB Towne, Hu Kennedy]","[United States Union of States, City of New Or...",[Constitution]
4,August 24 1865 APPLICATION AND CERTIFICATE OF ...,[JB Hiserodt],[Pass Christian Mississippi],"[WA Fleitas Amnesty Oath, US Cold Infty]"
5,I Wm Whaley solemnly swear I carefully read Am...,"[Wm Whaley, ANDREW JOHNSON, William Whaley, Le...","[United States of America, Charleston, SC, Cit...","[Amnesty Proclamation, Constitution]"
6,I J F Steinmeyer solemnly swear I carefully re...,"[I J F Steinmeyer, ANDREW JOHNSON, JF Steinmey...","[United States of America, Charleston Sc, Coun...","[Amnesty Proclamation, Constitution, Emancipat..."
7,I Joseph Edings solemnly swear I carefully rea...,"[Joseph Edings, ANDREW JOHNSON, Joseph Edings ...","[United States of America, County of Orangebur...","[Amnesty Proclamation, NYVV, Constitution]"
8,Vicksburg Miss July 22 1867 Thomas Andrew Oath...,[],[],[Thomas Andrew Oath of Loyalty Recd Bureau]
9,455 I Jacob Schroder solemnly swear I carefull...,"[Jacob Schroder, ANDREW JOHNSON, R A Bray, Alm...","[United States of America, UNITED STATES OF AM...","[Amnesty Proclamation, Walhalla, Constitution,..."


tagger1 ner english

In [24]:
lst=[]
label=[]
Per=[]
ORG=[]
LOC=[]
from flair.data import Sentence

dfner=pd.DataFrame({"text":"", "PERSON-nerenglish":"","LOC-nerenglish":"","ORG-nerenglish":[]})
for i in range(20):
        sentencess =Sentence(text[i])
        dfner.loc[i,"text"]=text[i]
        tagger1.predict(sentencess)
        dc={}
        P=[]
        L=[]
        O=[]
        for entity in sentencess.get_spans('ner'):
                     
                     if (entity.get_label("ner").score > 0.90) & (entity.get_label("ner").value != "CARDINAL") & (entity.get_label("ner").value != "LAW")& (entity.get_label("ner").value != "QUANTITY") & (entity.get_label("ner").value != "WORK_OF_ART") & (entity.get_label("ner").value != "LAW")& (entity.get_label("ner").value != "QUANTITY") & (entity.get_label("ner").value != "EVENT")  :
                        dc[entity.text]= entity.get_label("ner").value 
        
        
        for j in dc :
                if dc[j]=="PER":
                      P.append(j)
                elif (dc[j]=="ORG") | (dc[j]=="MISC") :
                      O.append(j)
                elif dc[j]=="LOC":
                      L.append(j)
               
        Per.append(P)
        ORG.append(O)
        LOC.append(L)
            

dfner["PERSON-nerenglish"]=Per
dfner["LOC-nerenglish"]=LOC
dfner["ORG-nerenglish"]=ORG                        

In [25]:
dfner.head()

,text,PERSON-nerenglish,LOC-nerenglish,ORG-nerenglish
0,8 E encl I Anne H Elliott widow solemnly swear...,"[Anne H Elliott, Andrew Johnson, Anne H]",[South Carolina],[]
1,I E C Whaley solemnly swear I carefully read A...,[Andrew Johnson],[],[]
2,43M I Mrs E Montgomery solemnly Swear I carefu...,[E Montgomery],[],[]
3,No 2324 I solemnly swear affirm presence of Al...,[],[],[]
4,August 24 1865 APPLICATION AND CERTIFICATE OF ...,[],[US],[WA Fleitas Amnesty Oath Administered At Pass ...


tagger 2 ontonotes

In [26]:
lst=[]
label=[]
Per=[]
Date=[]
GPE=[]
ORG=[]
from flair.data import Sentence

df=pd.DataFrame({"text":"", "PERSON-ontonotes":[], "GPE-ontonotes":"","ORG-ontonotes":"", "Date-ontonotes":""})
for i in range(20):
        sentencess =Sentence(text[i])
        df.loc[i,"text"]=text[i]
        tagger.predict(sentencess)
        dc={}
        P=[]
        D=[]
        G=[]
        O=[]
        for entity in sentencess.get_spans('ner'):
                     
                     if (entity.get_label("ner").score > 0.98) & (entity.get_label("ner").value != "CARDINAL") & (entity.get_label("ner").value != "LAW")& (entity.get_label("ner").value != "QUANTITY") & (entity.get_label("ner").value != "WORK_OF_ART") & (entity.get_label("ner").value != "LAW")& (entity.get_label("ner").value != "QUANTITY") & (entity.get_label("ner").value != "EVENT")  :
                        dc[entity.text]= entity.get_label("ner").value 
        
        
        for j in dc :
                if dc[j]=="PERSON":
                      P.append(j)
                elif dc[j]=="ORG":
                      O.append(j)
                elif dc[j]=="DATE":
                      D.append(j)
                elif dc[j]=="GPE":
                      G.append(j)
        Per.append(P)
        Date.append(D)
        ORG.append(O)
        GPE.append(G)
            
df["Date-ontonotes"]=Date
df["PERSON-ontonotes"]=Per
df["GPE-ontonotes"]=GPE
df["ORG-ontonotes"]=ORG    

In [27]:
df.head()

,text,PERSON-ontonotes,GPE-ontonotes,ORG-ontonotes,Date-ontonotes
0,8 E encl I Anne H Elliott widow solemnly swear...,"[Anne H Elliott, Andrew Johnson, S Wilde Durson]",[United States of America],[Amnesty],"[May 29th 1865, 27th day of Novb 1865, 27th da..."
1,I E C Whaley solemnly swear I carefully read A...,"[I E C Whaley, Andrew Johnson, E C Whaley, Lev...","[United State of America, Charleston SC, Unite...",[],"[May 29th 1865, 28 day of Sept 1865, Maj 47th ..."
2,43M I Mrs E Montgomery solemnly Swear I carefu...,"[E, ANDREW JOHNSON, Levi Stuber Major]","[United States of America, Charleston SC, Unit...",[Union of States],"[May 29th 1865, 31 day of Oct 1865, 61 year]"
3,No 2324 I solemnly swear affirm presence of Al...,[Hu Kennedy],[],[],[31st day of August AD 1865]
4,August 24 1865 APPLICATION AND CERTIFICATE OF ...,[JB Hiserodt],[US],[],[August 24 1865]


****

****

In [28]:
newdata.shape

(2136, 16)

In [29]:
all_text

0       I I C Meekins of Currituck County State of Nor...
1       Department of Virginia North Carolina 1864 tak...
2       I W H Pearce of Craven County State of NorthCa...
3       State of North Carolina County I solemnly swea...
4       State of North Carolina Wake County I SE Allen...
                              ...                        
2131    Office Provost Marshal Raleigh NC May 26 65 Th...
2132    July 30th 1867 Birnie Wm Agent Bu R F A L Oath...
2133    State of North Carolina Robeson County I Willi...
2134    NARA 449 102 We whose name written well person...
2135    NARA 450 404 Office Provost Marshall City of W...
Name: transcription_text, Length: 2136, dtype: object

# all dataset

In [30]:
lst=[]
label=[]
Per=[]
Date=[]
GPE=[]
ORG=[]
from flair.data import Sentence

aldf=pd.DataFrame({"text":"", "PERSON-ontonotes":[], "GPE-ontonotes":"","ORG-ontonotes":"", "Date-ontonotes":""})
for i in range(2136):
        sentencess =Sentence(all_text[i])
        aldf.loc[i,"text"]=all_text[i]
        tagger.predict(sentencess)
        dc={}
        P=[]
        D=[]
        G=[]
        O=[]
        for entity in sentencess.get_spans('ner'):
                     
                     if (entity.get_label("ner").score > 0.98) & (entity.get_label("ner").value != "CARDINAL") & (entity.get_label("ner").value != "LAW")& (entity.get_label("ner").value != "QUANTITY") & (entity.get_label("ner").value != "WORK_OF_ART") & (entity.get_label("ner").value != "LAW")& (entity.get_label("ner").value != "QUANTITY") & (entity.get_label("ner").value != "EVENT")  :
                        dc[entity.text]= entity.get_label("ner").value 
                        lst.append(entity)
        
        
        for j in dc :
                if dc[j]=="PERSON":
                      P.append(j)
                elif dc[j]=="ORG":
                      O.append(j)
                elif dc[j]=="DATE":
                      D.append(j)
                elif dc[j]=="GPE":
                      G.append(j)
        Per.append(P)
        Date.append(D)
        ORG.append(O)
        GPE.append(G)
            
aldf["Date-ontonotes"]=Date
aldf["PERSON-ontonotes"]=Per
aldf["GPE-ontonotes"]=GPE
aldf["ORG-ontonotes"]=ORG    

In [31]:
aldf.shape

(2136, 5)

In [32]:
aldf.head()

,text,PERSON-ontonotes,GPE-ontonotes,ORG-ontonotes,Date-ontonotes
0,I I C Meekins of Currituck County State of Nor...,"[I I C Meekins, Isaac C Meekins, E H Walker JP...",[North Carolina],[],[16 day of Sept AD 1865]
1,Department of Virginia North Carolina 1864 tak...,[],"[North Carolina, United States]",[Department of Virginia],[1864]
2,I W H Pearce of Craven County State of NorthCa...,"[W H Pearce, Watson]",[],[],[3rd day of October AD 1865]
3,State of North Carolina County I solemnly swea...,[],"[North Carolina County, United States]",[],[]
4,State of North Carolina Wake County I SE Allen...,[Allen],"[North Carolina, Wake County, United States]",[],[31st day of May 1867]


In [33]:
aldf["Date-ontonotes"]

0                                [16 day of Sept AD 1865]
1                                                  [1864]
2                            [3rd day of October AD 1865]
3                                                      []
4                                  [31st day of May 1867]
                              ...                        
2131                                    [May 26, July 14]
2132                                     [July 30th 1867]
2133                                                   []
2134    [May 29 1865, 15th Sept 1865, 15th September A...
2135    [Sept 22 1865, Twentyseventh day of September ...
Name: Date-ontonotes, Length: 2136, dtype: object

In [34]:
aldf.shape

(2136, 5)

# Post processing

**For dates**

In [35]:

G=[]
for i in df["Date-ontonotes"]:
    M=[]
    for j in i:
        date=re.search(r'^\d\.*.*\d\d$' ,j)
        date2=re.search(r'^[A-ZZa-z]\.*.*\d\.*.*\d\d$' ,j)
        M.append(date)
        M.append(date2)
    G.append(M)
    

GF=[]
for i in G:
    Final_date=[]
    for j in i:
        if j!=None:
            Final_date.append(j.group())
    GF.append(Final_date)
    
            
            
            

 

In [36]:
df["postDate-ontonotes"]=GF 

In [37]:
df.head()

,text,PERSON-ontonotes,GPE-ontonotes,ORG-ontonotes,Date-ontonotes,postDate-ontonotes
0,8 E encl I Anne H Elliott widow solemnly swear...,"[Anne H Elliott, Andrew Johnson, S Wilde Durson]",[United States of America],[Amnesty],"[May 29th 1865, 27th day of Novb 1865, 27th da...","[May 29th 1865, 27th day of Novb 1865, 27th da..."
1,I E C Whaley solemnly swear I carefully read A...,"[I E C Whaley, Andrew Johnson, E C Whaley, Lev...","[United State of America, Charleston SC, Unite...",[],"[May 29th 1865, 28 day of Sept 1865, Maj 47th ...","[May 29th 1865, 28 day of Sept 1865]"
2,43M I Mrs E Montgomery solemnly Swear I carefu...,"[E, ANDREW JOHNSON, Levi Stuber Major]","[United States of America, Charleston SC, Unit...",[Union of States],"[May 29th 1865, 31 day of Oct 1865, 61 year]","[May 29th 1865, 31 day of Oct 1865]"
3,No 2324 I solemnly swear affirm presence of Al...,[Hu Kennedy],[],[],[31st day of August AD 1865],[31st day of August AD 1865]
4,August 24 1865 APPLICATION AND CERTIFICATE OF ...,[JB Hiserodt],[US],[],[August 24 1865],[August 24 1865]


In [38]:
G=[]
for i in df["postDate-ontonotes"]:
    M=[]
    for j in i:
        Final_date1=re.sub(r'([0-9]{1,2})\.*.*\s([A-Z][a-z]\S*)\.*.*\s([0-9]{2,4})',r'\1-\2-\3' ,j)
        Final_date2=re.sub(r'([A-Z]\.*.*)\s([0-9]{1,2})\.*.*\s([0-9]{2,4})',r'\2-\1-\3' ,j)
        final_date3=re.sub(r'([A-Z][a-z]\S*)\.*.*\s([A-Z][a-z]\S*)\.*.*\s([0-9]{2,4})',r'\2-\1-\3' ,j)
        final_date3=re.sub(r'([A-Za-z][a-z]\S*.*.)day\.*.*\s([A-Z][a-z]\S*)\.*.*\s([0-9]{2,4})',r'\1-\2-\3' ,j)
        if Final_date1 != j:   
            M.append(Final_date1)
        if Final_date2 != j :
            M.append(Final_date2)
        if final_date3 != j :
            M.append(final_date3)
    G.append(M)
    

df["postDate-ontonotes"]=G

In [39]:
df.head()

,text,PERSON-ontonotes,GPE-ontonotes,ORG-ontonotes,Date-ontonotes,postDate-ontonotes
0,8 E encl I Anne H Elliott widow solemnly swear...,"[Anne H Elliott, Andrew Johnson, S Wilde Durson]",[United States of America],[Amnesty],"[May 29th 1865, 27th day of Novb 1865, 27th da...","[29-May-1865, 27-Novb-1865, 27th -Novb-1865, 2..."
1,I E C Whaley solemnly swear I carefully read A...,"[I E C Whaley, Andrew Johnson, E C Whaley, Lev...","[United State of America, Charleston SC, Unite...",[],"[May 29th 1865, 28 day of Sept 1865, Maj 47th ...","[29-May-1865, 28-Sept-1865]"
2,43M I Mrs E Montgomery solemnly Swear I carefu...,"[E, ANDREW JOHNSON, Levi Stuber Major]","[United States of America, Charleston SC, Unit...",[Union of States],"[May 29th 1865, 31 day of Oct 1865, 61 year]","[29-May-1865, 31-Oct-1865]"
3,No 2324 I solemnly swear affirm presence of Al...,[Hu Kennedy],[],[],[31st day of August AD 1865],"[31-August-1865, 31st -August-1865]"
4,August 24 1865 APPLICATION AND CERTIFICATE OF ...,[JB Hiserodt],[US],[],[August 24 1865],[24-August-1865]


# for the hole data


In [40]:
G=[]
for i in aldf["Date-ontonotes"]:
    M=[]
    for j in i:
        date=re.search(r'^\d\.*.*\d\d$' ,j)
        date2=re.search(r'^[A-ZZa-z]\.*.*\d\.*.*\d\d$' ,j)
        M.append(date)
        M.append(date2)
    G.append(M)
    

GF=[]
for i in G:
    Final_date=[]
    for j in i:
        if j!=None:
            Final_date.append(j.group())
    GF.append(Final_date)

    
aldf["postDate-ontonotes"]=GF
            
            
        

In [41]:
aldf.shape

(2136, 6)

In [42]:
G=[]
for i in aldf["postDate-ontonotes"]:
    M=[]
    for j in i:
        Final_date1=re.sub(r'([0-9]{1,2})\.*.*\s([A-Z][a-z]\S*)\.*.*\s([0-9]{2,4})',r'\1-\2-\3' ,j)
        Final_date2=re.sub(r'([A-Z]\.*.*)\s([0-9]{1,2})\.*.*\s([0-9]{2,4})',r'\2-\1-\3' ,j)
        final_date3=re.sub(r'([A-Z][a-z]\S*)\.*.*\s([A-Z][a-z]\S*)\.*.*\s([0-9]{2,4})',r'\1-\2-\3' ,j)
        if Final_date1 != j:   
            M.append(Final_date1)
        if Final_date2 != j :
            M.append(Final_date2)
        if final_date3 != j :
            M.append(final_date3)
    G.append(M)
    

aldf["postDate-ontonotes"]=G

In [43]:
aldf.shape

(2136, 6)

In [44]:
aldf["postDate-ontonotes"]

0                                          [16-Sept-1865]
1                                                      []
2                                        [3-October-1865]
3                                                      []
4                                           [31-May-1867]
                              ...                        
2131                                                   []
2132                                       [30-July-1867]
2133                                                   []
2134    [29-May-1865, 15-Sept-1865, 15-September-1865,...
2135    [22-Sept-1865, Twentyseventh-September-1865, 2...
Name: postDate-ontonotes, Length: 2136, dtype: object

**great !**

# best result obtained 

***Person***

****

***Organization***

****

***GPE and location***

****

**Date**

****

In [45]:
df.head()

,text,PERSON-ontonotes,GPE-ontonotes,ORG-ontonotes,Date-ontonotes,postDate-ontonotes
0,8 E encl I Anne H Elliott widow solemnly swear...,"[Anne H Elliott, Andrew Johnson, S Wilde Durson]",[United States of America],[Amnesty],"[May 29th 1865, 27th day of Novb 1865, 27th da...","[29-May-1865, 27-Novb-1865, 27th -Novb-1865, 2..."
1,I E C Whaley solemnly swear I carefully read A...,"[I E C Whaley, Andrew Johnson, E C Whaley, Lev...","[United State of America, Charleston SC, Unite...",[],"[May 29th 1865, 28 day of Sept 1865, Maj 47th ...","[29-May-1865, 28-Sept-1865]"
2,43M I Mrs E Montgomery solemnly Swear I carefu...,"[E, ANDREW JOHNSON, Levi Stuber Major]","[United States of America, Charleston SC, Unit...",[Union of States],"[May 29th 1865, 31 day of Oct 1865, 61 year]","[29-May-1865, 31-Oct-1865]"
3,No 2324 I solemnly swear affirm presence of Al...,[Hu Kennedy],[],[],[31st day of August AD 1865],"[31-August-1865, 31st -August-1865]"
4,August 24 1865 APPLICATION AND CERTIFICATE OF ...,[JB Hiserodt],[US],[],[August 24 1865],[24-August-1865]


In [46]:
dflarge.head()

,text,PERSON-nerenglarge,LOC-nerenglarge,ORG-nerenglarge
0,8 E encl I Anne H Elliott widow solemnly swear...,"[Anne H Elliott, Andrew Johnson, S Wilde Durso...","[United States of America, District of Beaufor...","[Amnesty, Constitution, Emancipation of Slaves]"
1,I E C Whaley solemnly swear I carefully read A...,"[I E C Whaley, Andrew Johnson, E C Whaley, Lev...","[United State of America, United States of Ame...","[Amnesty Proclamation, Constitution, Emancipat..."
2,43M I Mrs E Montgomery solemnly Swear I carefu...,"[Mrs E Montgomery, ANDREW JOHNSON, Sgd Montgom...","[United States of America, Charleston, SC, Cou...","[Amnesty Proclamation, 47th Pa, Vols, Emancipa..."
3,No 2324 I solemnly swear affirm presence of Al...,"[God, GOD, EB Towne, Hu Kennedy]","[United States Union of States, City of New Or...",[Constitution]
4,August 24 1865 APPLICATION AND CERTIFICATE OF ...,[JB Hiserodt],[Pass Christian Mississippi],"[WA Fleitas Amnesty Oath, US Cold Infty]"


In [47]:
dfner.head()

,text,PERSON-nerenglish,LOC-nerenglish,ORG-nerenglish
0,8 E encl I Anne H Elliott widow solemnly swear...,"[Anne H Elliott, Andrew Johnson, Anne H]",[South Carolina],[]
1,I E C Whaley solemnly swear I carefully read A...,[Andrew Johnson],[],[]
2,43M I Mrs E Montgomery solemnly Swear I carefu...,[E Montgomery],[],[]
3,No 2324 I solemnly swear affirm presence of Al...,[],[],[]
4,August 24 1865 APPLICATION AND CERTIFICATE OF ...,[],[US],[WA Fleitas Amnesty Oath Administered At Pass ...


In [48]:
df["Person-NerEngLarge"]=dflarge["PERSON-nerenglarge"]
df["LOC-NerEngLarge"]=dflarge["LOC-nerenglarge"]
df["ORG-NerEngLarge"]=dflarge["ORG-nerenglarge"]
df["Person-NerEng"]=dfner["PERSON-nerenglish"]
df["LOC-NerEng"]=dfner["LOC-nerenglish"]
df["ORG-NerEng"]=dfner["ORG-nerenglish"]

In [49]:
df.head()

,text,PERSON-ontonotes,GPE-ontonotes,ORG-ontonotes,Date-ontonotes,postDate-ontonotes,Person-NerEngLarge,LOC-NerEngLarge,ORG-NerEngLarge,Person-NerEng,LOC-NerEng,ORG-NerEng
0,8 E encl I Anne H Elliott widow solemnly swear...,"[Anne H Elliott, Andrew Johnson, S Wilde Durson]",[United States of America],[Amnesty],"[May 29th 1865, 27th day of Novb 1865, 27th da...","[29-May-1865, 27-Novb-1865, 27th -Novb-1865, 2...","[Anne H Elliott, Andrew Johnson, S Wilde Durso...","[United States of America, District of Beaufor...","[Amnesty, Constitution, Emancipation of Slaves]","[Anne H Elliott, Andrew Johnson, Anne H]",[South Carolina],[]
1,I E C Whaley solemnly swear I carefully read A...,"[I E C Whaley, Andrew Johnson, E C Whaley, Lev...","[United State of America, Charleston SC, Unite...",[],"[May 29th 1865, 28 day of Sept 1865, Maj 47th ...","[29-May-1865, 28-Sept-1865]","[I E C Whaley, Andrew Johnson, E C Whaley, Lev...","[United State of America, United States of Ame...","[Amnesty Proclamation, Constitution, Emancipat...",[Andrew Johnson],[],[]
2,43M I Mrs E Montgomery solemnly Swear I carefu...,"[E, ANDREW JOHNSON, Levi Stuber Major]","[United States of America, Charleston SC, Unit...",[Union of States],"[May 29th 1865, 31 day of Oct 1865, 61 year]","[29-May-1865, 31-Oct-1865]","[Mrs E Montgomery, ANDREW JOHNSON, Sgd Montgom...","[United States of America, Charleston, SC, Cou...","[Amnesty Proclamation, 47th Pa, Vols, Emancipa...",[E Montgomery],[],[]
3,No 2324 I solemnly swear affirm presence of Al...,[Hu Kennedy],[],[],[31st day of August AD 1865],"[31-August-1865, 31st -August-1865]","[God, GOD, EB Towne, Hu Kennedy]","[United States Union of States, City of New Or...",[Constitution],[],[],[]
4,August 24 1865 APPLICATION AND CERTIFICATE OF ...,[JB Hiserodt],[US],[],[August 24 1865],[24-August-1865],[JB Hiserodt],[Pass Christian Mississippi],"[WA Fleitas Amnesty Oath, US Cold Infty]",[],[US],[WA Fleitas Amnesty Oath Administered At Pass ...


In [50]:
df.to_csv("result.csv", index=False)

In [51]:
aldf.head()

,text,PERSON-ontonotes,GPE-ontonotes,ORG-ontonotes,Date-ontonotes,postDate-ontonotes
0,I I C Meekins of Currituck County State of Nor...,"[I I C Meekins, Isaac C Meekins, E H Walker JP...",[North Carolina],[],[16 day of Sept AD 1865],[16-Sept-1865]
1,Department of Virginia North Carolina 1864 tak...,[],"[North Carolina, United States]",[Department of Virginia],[1864],[]
2,I W H Pearce of Craven County State of NorthCa...,"[W H Pearce, Watson]",[],[],[3rd day of October AD 1865],[3-October-1865]
3,State of North Carolina County I solemnly swea...,[],"[North Carolina County, United States]",[],[],[]
4,State of North Carolina Wake County I SE Allen...,[Allen],"[North Carolina, Wake County, United States]",[],[31st day of May 1867],[31-May-1867]


In [52]:
newdata.shape

(2136, 16)

In [53]:
allPerson=[]
for i in aldf["PERSON-ontonotes"]:
            allPerson.append(i)
allDatess=[]
for i in aldf["Date-ontonotes"]:
            allDatess.append(i)
allpostdates=[]
for i in aldf["postDate-ontonotes"]:
            allpostdates.append(i)
allGPE=[]
for i in aldf["GPE-ontonotes"]:
            allGPE.append(i)
alORGl=[]
for i in aldf["ORG-ontonotes"]:
            alORGl.append(i)

In [54]:
allpostdates

[['16-Sept-1865'],
 [],
 ['3-October-1865'],
 [],
 ['31-May-1867'],
 ['1-Dec-1866'],
 ['30-May-1864'],
 ['16-Sept-1865'],
 ['16-Nov-1866'],
 ['16-Sept-1865'],
 ['4-November-1867'],
 ['4-November-1867'],
 ['2-Oct-1865'],
 [],
 ['31-December-1867'],
 ['19-June-1865', '19-June-1865'],
 ['4-Nov-1865'],
 ['The 9-November-1866', '9-The-1866', 'The-November-1866'],
 ['1-December-1866', '1-Dec-1866'],
 ['25-June-1864'],
 ['20-October-1866'],
 ['Third-December-1866'],
 ['Twenty-July-1867'],
 ['19-May-1865', 'Nineteenth-May-1865'],
 [],
 [],
 ['15-Janry-1868'],
 ['1-July-1867'],
 [],
 [],
 [],
 ['3-Aug-67'],
 ['3-August-1867'],
 ['15-Dec-67'],
 [],
 [],
 [],
 ['15-Apr-67'],
 [],
 ['23-July-67'],
 ['23-July-1867'],
 ['26-July-1867'],
 ['25-July-1867'],
 [],
 [],
 [],
 [],
 [],
 [],
 ['29-Aug-66'],
 [],
 ['1-Nov-1866'],
 [],
 ['9-Nov-66'],
 [],
 [],
 ['26-July-1867'],
 ['31-July-1868'],
 [],
 ['18-Jany-1868'],
 [],
 ['14-Jany-1868'],
 [],
 ['14-Jany-1868'],
 [],
 ['2-Aug-67'],
 [],
 ['1-May-67'],
